# 목표: DACON Baseline 코드를 이용해 LSTM을 활용한 학습 진행

In [1]:
import tensorflow as tf
# tf.__version__
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config=config....)

# physical_devices = tf.config.list_physical_devices('GPU') 
# for device in physical_devices:
#     tf.config.experimental.set_memory_growth(device, True)


# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     # 텐서플로가 첫 번째 GPU에 1GB 메모리만 할당하도록 제한
#     try:
#         tf.config.experimental.set_virtual_device_configuration(
#             gpus[0],
#             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
#     except RuntimeError as e:
#         # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
#         print(e)
    

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

2021-12-06 16:22:57.166578: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
tf.config.experimental.list_physical_devices('GPU')
# physical_devices = tf.config.list_physical_devices('GPU') 
# for device in physical_devices:
#     tf.config.experimental.set_memory_growth(device, True)

2021-12-06 16:23:00.974155: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-12-06 16:23:01.004871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-06 16:23:01.005438: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:1e.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2021-12-06 16:23:01.005462: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-12-06 16:23:01.007275: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-12-06 16:23:01.008482: I tensorflow/stream_executor/platform/default/d

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# 목표: DACON Baseline 코드를 이용해 LSTM을 활용한 학습 진행

import json
import numpy as np
import pandas as pd

import io

import os
import matplotlib.image as img
import matplotlib.pyplot as plt
from tqdm import tqdm

# import tensorflow as tf
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# from scipy import stats

In [6]:
# train=pd.read_csv('train_features.csv')
# train_labels=pd.read_csv('train_labels.csv')
# test=pd.read_csv('test_features.csv')
# submission=pd.read_csv('sample_submission.csv')

In [7]:
from numpy import genfromtxt

train_x_data_DF = genfromtxt('./train/train_x.csv', delimiter=',')
train_t_mod_DF = pd.read_csv('./train/train_t.csv', sep=',')

valid_x_data_DF = genfromtxt('./train/valid_x.csv', delimiter=',')
valid_t_mod_DF = pd.read_csv('./train/valid_t.csv', sep=',')
# print(train_x_data_DF)
# print(train_x_data_DF.shape)
# print(train_t_mod_DF)

train_x_data = train_x_data_DF
train_t_mod = train_t_mod_DF

valid_x_data = valid_x_data_DF
valid_t_mod = valid_t_mod_DF

print(train_x_data)
print(train_x_data.shape)
print(type(train_x_data[0][0]))
print(train_t_mod.shape)

print(valid_x_data)
print(valid_x_data.shape)
print(type(valid_x_data[0][0]))
print(valid_t_mod.shape)

[[ 940.  648.  958. ...  414. 1044.  470.]
 [ 940.  643.  966. ...  414. 1079.  479.]
 [1014.  628. 1005. ...  413. 1061.  469.]
 ...
 [ 904.  963.  929. ...  508. 1013.  683.]
 [ 904.  963.  929. ...  508. 1013.  683.]
 [ 904.  963.  929. ...  508. 1013.  683.]]
(97904, 26)
<class 'numpy.float64'>
(12238, 1)
[[ 920.  844.  915. ...  549.  821.  512.]
 [ 920.  844.  901. ...  587.  790.  643.]
 [ 746.  844.  784. ...  562.  798.  627.]
 ...
 [1113.  597. 1105. ...  324.  990.  394.]
 [1120.  602. 1106. ...  338.  991.  396.]
 [1120.  602. 1127. ...  344. 1026.  408.]]
(41960, 26)
<class 'numpy.float64'>
(5245, 1)


In [8]:
import easydict
 
args = easydict.EasyDict({
        "batchsize": 1,
        "epoch": 1,
#         "gpu": 0,
        "out": "result",
        "resume": False,
#         "unit": 1000,
        "validationsplit": 0.2,
        "optimizer": "rmsprop",
        "frame": 8,
#         "optimizer": "Adam",
        
})

In [10]:
# [ -1, (한번에 들어갈 입력값 수), (좌표개수) ]
# 13개의 x,y 좌표이므로 26개
X=tf.reshape(train_x_data,[-1, args.frame, 26])
X.shape

2021-12-06 16:23:38.798024: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-06 16:23:38.823119: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2499995000 Hz
2021-12-06 16:23:38.823359: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55dc00205ab0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-12-06 16:23:38.823382: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-12-06 16:23:38.921229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA nod

TensorShape([12238, 8, 26])

In [11]:
train_t_mod.shape
# print(type(train_t_mod[0]))
# print(type(train_x_data[0][0]))

(12233, 1)

In [12]:
y = tf.reshape(train_t_mod,[-1,1])
y.shape

TensorShape([12233, 1])

In [13]:
#가벼운 모델 생성
model = Sequential()
model.add(LSTM(32, input_shape=(args.frame,26)))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(61, activation='softmax')) # 다중 분류
model.add(Dense(1, activation='sigmoid'))

# Adam 최적화함수를 사용해보는 것도 고려해볼 것
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy']) # 다중 분류
model.compile(optimizer=args.optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X,
          y, epochs=args.epoch, batch_size=args.batchsize, validation_split=0.2)

2021-12-06 11:44:35.265872: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-12-06 11:44:35.266555: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-12-06 11:44:35.266847: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-12-06 11:44:35.266875: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-12-06 11:44:35.287241: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2021-12-06 11:44:36.787962: E tensorflow/stream_executor/cuda/cuda_blas.cc:225] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-12-06 11:44:36.787994: W tensorflow/stream_executor/stream.cc:2055] attempting to perform BLAS operation using StreamExecutor without BLAS su

InternalError:  Blas GEMV launch failed:  m=32, n=1
	 [[node sequential/dense/MatMul (defined at tmp/ipykernel_4592/1671018923.py:2) ]] [Op:__inference_train_function_2647]

Function call stack:
train_function


In [ ]:
test_X=tf.reshape(test_x_data,[-1, args.frame, 26])
test_X.shape

In [ ]:
prediction=model.predict(test_X)

In [ ]:
prediction.shape